# Legacy vs Aplac HB use

There is still significant legacy HB use and it is desirable to understand this use more.  These are the users that have used only the legacy HB engine with the number of user-weeks in which they have used it.

Analysis date: **August 27, 2017**

In [31]:
for w, u in sorted(users, key=lambda x: x[0], reverse=True):
    print(u)

MA043304 @ MA-Com at MACOM Technology Solutions, Inc. - MA, weeks=24
id=4110 at Friends, weeks=13
Hailing Yue? at Qorvo FL at Qorvo - FL, weeks=11
Maxime Malnou at University of Colorado at Boulder - Department of Electrical Engineering, weeks=9
Claudio Jarufe at Universidad de Chile, weeks=9
Claudio Jarufe at Universidad de Chile, weeks=9
id=1019 at Exodus Advanced Communications, weeks=8
Adam @ Spaceforest at Spaceforest, weeks=7
Bryce Hotalen @ Qorvo FL at Qorvo - FL, weeks=7
Idan Peretz at Technion Israel Institute of Technology - Engineering Department Communications Lab, weeks=7
id=3643 at Qorvo/RF Micro Devices, Inc. - Greensboro, weeks=7
John Stensby at University of Alabama in Huntsville, weeks=7
Didier Balfet at Sigfox, weeks=7
sasha rajcevich at University of Utah, weeks=6
Kazuki Ishihara at Sophia University, weeks=6
Xufeng Du at University of Technology Dresden, weeks=6
Edan Alpern at University of Pittsburgh, weeks=6
koffi Cornelis at DEV TECNOLOGIA INDUSTRIA, COMERCIO E 

----

# Analysis

In [15]:
datadir = '/Users/dane/data/a/'
import pandas as pd
# featured data only includes external users
df = pd.read_csv(datadir + 'tlm/features/featured_user_weeks.csv', encoding='latin1')

# remove all the internal use
df = df[df.user_type != 'internal']
len(df)

13642

In [16]:
df.columns

Index(['Unnamed: 0', 'HB_aplac_b', 'HB_legacy_b', 'add_measurement_b',
       'add_mwo_elem_b', 'add_sim_doc_b', 'add_vss_elem_b', 'ana_doc_b',
       'ana_sim_n', 'antenna_b', 'board_import_b', 'cell_stretchers_b',
       'commands_n', 'company', 'disp_name', 'ele_created_n', 'em_sim_b',
       'ems_doc_b', 'ems_highcnt_b', 'ems_sim_n', 'enable_disable_b',
       'extract_b', 'freeze_traces_b', 'gen_poly_b', 'ifilter_b', 'layout_b',
       'layout_checks_b', 'lin_noise_b', 'lin_sim_n', 'load_pull_b',
       'markers_b', 'measurements_n', 'no_sim_b', 'nonlin_sim_n', 'optim_b',
       'osc_b', 'output_eq_b', 'pdk_b', 'schematic_eq_b', 'script_ide_b',
       'scripts_b', 'stability_b', 'symbol_gen_b', 'transient_b', 'tuning_b',
       'user_id', 'user_type', 'vss_doc_b', 'vss_rfb_b', 'vss_rfi_b',
       'vss_rfp_b', 'vss_sim_n', 'week', 'xem_doc_b', 'xem_sim_n', 'xmodels_b',
       'yield_b', 'activity', 'scope'],
      dtype='object')

In [17]:
# collect up HB counts, sum gives number of weeks
all_hb = df[['user_id', 'HB_legacy_b', 'HB_aplac_b']].groupby('user_id').sum()

# remove empty
mask = (all_hb.HB_legacy_b == 0) & (all_hb.HB_aplac_b == 0)
with_hb = all_hb[~mask]

In [18]:
mask_l = with_hb.HB_legacy_b != 0
mask_a = with_hb.HB_aplac_b != 0
print( 'Users using only legacy HB', len(with_hb[mask_l & ~mask_a]))
print( 'Users using only Aplac HB', len(with_hb[~mask_l & mask_a]))
print( 'Users using both', len(with_hb[mask_l & mask_a]))

Users using only legacy HB 324
Users using only Aplac HB 127
Users using both 213


We can look up the users that are using only Legacy HB to get more info about them. Let's only look at those that have used legacy HB in 4 or more weeks.

In [28]:
# let's look at these legacy only users
legacy = with_hb[mask_l & ~mask_a]
big_users = legacy[legacy.HB_legacy_b > 3].index  # used in 4 weeks or more
users = []
for user in big_users:
    n = 'nan'
    x = df[(df.user_id == user) & (df.HB_legacy_b > 0)]
    u = x.iloc[0]
    try:
        n = u.disp_name or u.start_user
    except:
        pass
    if str(n) == 'nan':
        n = "id=" + str(u.user_id)
    users.append([len(x), '{} at {}, weeks={}'.format(n, u.company, len(x))])